In [1]:
import yaml 
import pandas as pd
import os
from collections import Counter
import numpy as np
import torch
import tqdm
import pickle
from sklearn.metrics import accuracy_score
import evaluate

In [2]:
config_path = "../configs/pcgu_config.yaml"
with open(config_path, "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)

In [3]:
metric = evaluate.load(config["validation_metric"])

Using the latest cached version of the module from /ist/users/sitipornl/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Mon Aug  7 19:14:12 2023) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


In [4]:
biased_label_maps = {"entailment": 0, "contradiction": 1, "neutral": 2}
main_label_maps = {"entailment": 0, "contradiction": 1, "neutral": 2}
biased_label_remaps = {v:k for k,v in biased_label_maps.items()}
main_label_remaps   = {v:k for k,v in main_label_maps.items()}
data_path  = config["data_path"]
train_data = config["train_data"]
data_path = os.path.join('../',data_path, train_data)
biased_df = pd.read_json(data_path, lines=True)
seed = str(config['seed'])

predictions = []
results = []

# ************* Biased model **************
for index, row in biased_df.iterrows():
    prediction =  biased_label_remaps[int(torch.argmax(torch.Tensor(row['bias_probs']), dim=0))]
    predictions.append(prediction)
    # results.append(prediction  == "entailment")
    results.append(prediction  == row['gold_label'])

biased_df['predictions'] = predictions
biased_df['results'] = results
biased_df['gold_label_ids'] = biased_df['gold_label'].apply(lambda row : biased_label_maps[row])
biased_df['prediction_ids'] = biased_df['predictions'].apply(lambda row : biased_label_maps[row])

print(f"Bias model acc : {metric.compute(predictions=biased_df['prediction_ids'].tolist() , references=biased_df['gold_label_ids'].tolist() ) }")

Bias model acc : {'accuracy': 0.4386812392093751}


In [5]:
all_acc = 0
print(f'Bias model performance')
for label_text in biased_label_maps.keys():
    mask = biased_df['gold_label'] == label_text
    predictions = biased_df['prediction_ids'][mask].tolist()
    references  = biased_df['gold_label_ids'][mask].tolist()
    cur_acc = metric.compute(predictions=predictions , references=references)
    print(f"{label_text} class : {cur_acc}")
    all_acc += cur_acc['accuracy']
print(f'Over all acc : {all_acc / len(biased_label_maps.keys())}')

Bias model performance
entailment class : {'accuracy': 0.5876515481401692}
contradiction class : {'accuracy': 0.34619527436346}
neutral class : {'accuracy': 0.3822001527883881}
Over all acc : 0.43868232509733907
